In [ ]:
import os
import numpy as np
import open3d as o3d
from fns import center_vertices, normalize_vertices_scale, quantize_verts, dequantize_verts, prepare_halfedge_mesh
from utils.utils import SaveOBJ

In [ ]:
OUTPUT_DIR="./output"

if not os.path.exists("./output") :
  os.mkdir("./output")

In [ ]:
MESH_PATH="./demo/NewMesh1_Tri.obj"


In [ ]:
#--- Load and normalize mesh
mesh = o3d.io.read_triangle_mesh(MESH_PATH)

coordFrame= o3d.geometry.TriangleMesh.create_coordinate_frame()
rotation=coordFrame.get_rotation_matrix_from_xyz((np.pi/2, -np.pi/2, 0))
mesh.rotate(rotation, center=(0, 0, 0))

vertices = np.asarray(mesh.vertices)
vertices = center_vertices(vertices)
vertices = normalize_vertices_scale(vertices)
vertices = np.clip(vertices, a_min=-0.5, a_max = 0.5)
triangles = np.asarray(mesh.triangles)

#Quantize/dequantize vertices
o3d.io.write_triangle_mesh(OUTPUT_DIR+"/"+"BeforeQuantize_"+os.path.split(MESH_PATH)[1], mesh)
vertices=quantize_verts(vertices, 7)
vertices=dequantize_verts(vertices, 7)
o3d.io.write_triangle_mesh(OUTPUT_DIR+"/"+"AfterQuantize_"+os.path.split(MESH_PATH)[1], mesh)

mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(triangles)
mesh.remove_duplicated_vertices()
mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()
mesh.remove_unreferenced_vertices()

#Debug : save normalized mesh
o3d.io.write_triangle_mesh(OUTPUT_DIR+"/"+"normalized_"+os.path.split(MESH_PATH)[1], mesh)

#--- Sort mesh elements
sortedHalfedgeMesh, _, _ = prepare_halfedge_mesh(np.asarray(mesh.vertices), np.asarray(mesh.triangles))

#Debug : save sorted mesh
o3d.io.write_triangle_mesh(OUTPUT_DIR+"/"+"sorted_"+os.path.split(MESH_PATH)[1], mesh)
SaveOBJ(mesh.vertices, mesh.triangles, OUTPUT_DIR+"/"+"sortedColored_"+os.path.split(MESH_PATH)[1]+".obj", True)

# #--- Extract boundary to remesh
# submesh, remeshBoundary, sampledPoints=GenerateAreaToRemesh(mesh, TRIANGLES_TO_REMESH, iMaxRingSize=CONTEXT_RING, iNbSamples=NB_SAMPLING_POINTS)

# #Debug : Save area to remesh in OBJ
# SaveAreaToRemeshInOBJ(submesh, remeshBoundary, OUTPUT_DIR+"/"+"area2remesh_"+os.path.split(MESH_PATH)[1], iSampledPoints=sampledPoints)
# o3d.io.write_point_cloud(OUTPUT_DIR+"/"+"Sampling_"+os.path.split(MESH_PATH)[1]+".xyz", sampledPoints)

# #Check number of faces
# print("Number of faces to remesh: ", len(submesh.triangles))
# if len(submesh.triangles) >= DECIMATION_TARGET_NFACES:
#     raise Exception("@@@@ Number of faces to remesh is larger than target number of faces")